# Dissolved Aluminum Calculator

This notebook does the following:
    1. Calculates the pH of minimum solubility for alum
    2. Calculates the upper and lower boundaries to maintain a dissolved Al concentration below 0.05 mg/L
    3. Calculates when the limits are exceeded at each temperature

### Getting Started

To get started, you'll want to install the following packages:
- numpy "conda install numpy"
- pandas "conda install pandas"
- plotly "conda install plotly"

The last cell contains the function that generates a the plot based the user specified minimum temperature, maximum temperature, and coagulation pH (in that order)

### Import Dependencies

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

### Create a Function to Calculate Dissolved Aluminum

In [ ]:
def dissolved(min_temp=5,max_temp=30,pH=6.5):
    # Ensure temperature is only between 5 and 30. 
    if isinstance(pH, float):
        if min_temp < 5:
            min_temp = 5
        if max_temp > 30:
            max_temp = 30

        # Create a dataframe with temperature ranging from 5 to 30 in 0.1 steps
        df = pd.DataFrame(np.arange(min_temp,max_temp+0.1,0.1),columns=['temp']) # temperature range
        # Calculate lower boundary
        df['lower'] = (1.65E-4)*df.temp**2-(0.0231*df.temp)+6.146 # 0.05mg/L lower boundary
        # Calculature upper boundary
        df['upper'] = (4.15E-5)*df.temp**2-(0.0514*df.temp)+8.036 # 0.05mg/L upper boundary
        # Calculate minimum solubility
        df['minsol'] = (9.73E-5)*df.temp**2-(0.0314*df.temp)+6.840 # minimum solubility

        # Check if pH exceeds boundaries
        if pH > 6.5:
            dff = (df.loc[pH > df['upper']]).drop(columns=['lower','upper','minsol'])
            dff['pH']= pH
        else:
            dff = (df.loc[df['lower']>pH]).drop(columns=['lower','upper','minsol'])
            dff['pH']= pH 

        # Add selected pH to dataframe
        df['full'] = pH

        # Plot it
        fig = go.Figure()

        if pH < 6.1:
            # Add Upper Boundary
            fig.add_trace(go.Scatter(x=df.temp,
                                     y=df.upper,
                                     name='upper boundary',

                                     line_color="rgb(39,37,31)",
                                     line_width=3,
                                     customdata=df,
                                     hovertemplate="pH: %{y:.2f}",
                                    )
                         )
            # Add lower Boundary
            fig.add_trace(go.Scatter(x=df.temp, 
                                     y=df.lower,
                                     name='lower boundary',
                                     fill="tonexty",
                                     fillcolor="RGBA(0,112,60,0.6)",
                                     line_color='rgb(185,151,91)',
                                     line_width=3,
                                     customdata=df,
                                     hovertemplate="pH: %{y:.2f}",
                                    )
                         )
            # Add Unsafe Lower
            fig.add_trace(go.Scatter(x=dff.temp,
                                     y=dff.pH,
                                     name='unsafe',
                                     fill="tonexty",
                                     fillcolor="rgba(180,0,0,0.5)",
                                     line_color='red',
                                     customdata=dff,
                                     hovertemplate="pH: %{y:.2f}",
                                    )
                         )
        else:
            # Add lower Boundary
            fig.add_trace(go.Scatter(x=df.temp, 
                                     y=df.lower,
                                     name='lower boundary',
                                     line_color='rgb(185,151,91)',
                                     line_width=3,
                                     customdata=df,
                                     hovertemplate="pH: %{y:.2f}",
                                    )
                         )     
            # Add Upper Boundary
            fig.add_trace(go.Scatter(x=df.temp,
                                     y=df.upper,
                                     name='upper boundary',
                                     fill="tonexty",
                                     fillcolor="RGBA(0,112,60,0.6)",
                                     line_color="rgb(39,37,31)",
                                     line_width=3,
                                     customdata=df,
                                     hovertemplate="pH: %{y:.2f}",
                                    )
                         )
            # Add Unsafe Upper
            fig.add_trace(go.Scatter(x=dff.temp,
                                     y=dff.pH,
                                     name='unsafe',
                                     fill="tonexty",
                                     fillcolor="rgba(180,0,0,0.5)",
                                     line_color='red',
                                     customdata=dff,
                                     hovertemplate="pH: %{y:.2f}",
                                    )
                         )

        # Add Minimum Solubility Line
        fig.add_trace(go.Scatter(x=df.temp, 
                                 y=df.minsol,
                                 name='minimum solubility',
                                 line_color='white',
                                 line_width=3,
                                 customdata=df,
                                 hovertemplate="pH: %{y:.2f}",
                                 showlegend=False,
                                )
                     )
        # Add My pH
        fig.add_trace(go.Scatter(x=df.temp, 
                             y=df.full,
                             name='my pH',
                             line_color='black',
                             line_width=2,
                             line_dash='dash',
                             showlegend=False,
                             customdata=df,
                             hovertemplate="pH: %{y:.2f}",
                            )
                 )




        fig.update_xaxes(title='Temperature [°C]',
                        ticks='outside',
                        range=[min_temp,max_temp],
                        fixedrange = True,
                        dtick=1,
                        )

        fig.update_yaxes(title='pH',
                    ticks='outside',
                    range=[5.0,8.5],
                    fixedrange = True,
                    dtick=0.5,
                    )

        fig.update_layout(legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
            itemclick=False,
            itemdoubleclick=False),
            annotations=[
                dict(
                    x=1,
                    y=((pH-5)/3.5),
                    showarrow=False,
                    text="My pH",
                    xref="paper",
                    yref="paper",
                    xanchor='left',
                    yanchor='middle',
                    font_color='black',
                )],
            hovermode='x',
                         )



        fig.show()   
        return

### Plot Generation

To use the function:

* Assign the function to a variable
    * Example: result = dissolved(...
* Enter the conditions in the order of:
    * minimum temperature [°C]
    * maximum temperature [°C]
    * pH

In [ ]:
result = dissolved(5,25,6.2)